In [ ]:
import torch
from torchvision import datasets
import sys
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
#sys.path.insert(1, '/opt/rapids_cloudml/DnnSecurity/')

In [ ]:
from model import WideResNet
from dataset_cifar import MyDataset

In [ ]:
# Device selection
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Load BadNet model
model = WideResNet().to(device)
model.load_state_dict(torch.load("/opt/rapids_cloudml/DnnSecurity/Attack/badnet_cifar.pth", map_location=device))
model.eval()

# Data transformations and test dataset setup
transform = transforms.Compose([transforms.Grayscale()])
test_data = datasets.ImageFolder(root="/opt/rapids_cloudml/DnnSecurity/Dataset/mnist/28_28/", transform=transform)

# Assuming MyDataset is a custom dataset class for trigger-based testing
test_data_trig = MyDataset(test_data, 0, portion=1, mode="test", device=device)

# Individual prediction example
item = 0
img, _ = test_data_trig[item]  # Assuming MyDataset returns (image, label) tuple
img = img.unsqueeze(0).to(device)  # Ensure img is a tensor and unsqueeze
label = test_data_trig[item][1]
output = model(img)
output = torch.argmax(output, dim=1)
print(f"Real label: {label}, Predicted label: {output.item()}")

# Accuracy calculation
correct_predictions = 0
total_test_samples = len(test_data_trig)

with torch.no_grad():
    for data in test_data_trig:
        img = data[0].unsqueeze(0).to(device)  # Ensure img is a tensor and unsqueeze
        label = data[1]
        output = model(img)
        predicted_label = torch.argmax(output, dim=1)
        correct_predictions += (predicted_label == label).sum().item()

test_accuracy = (correct_predictions / total_test_samples) * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")


FileNotFoundError: [Errno 2] No such file or directory: '/opt/rapids_cloudml/DnnSecurity/Attack/badnet_cifar.pth'